# Import

In [4]:
pip install kagglehub[pandas-datasets]

In [5]:
# Dasar
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import re

# Modling Dan Evaluasi
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load Data

In [6]:
# Set the path to the file you'd like to load
file_path = "job_descriptions.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ravindrasinghrana/job-description-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-6-8c4b047a5b8b>:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 457M/457M [00:22<00:00, 21.3MB/s]

Extracting zip of job_descriptions.csv...


# EDA

Jumlah Keseluruhan Data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Job Id            1615940 non-null  int64  
 1   Experience        1615940 non-null  object 
 2   Qualifications    1615940 non-null  object 
 3   Salary Range      1615940 non-null  object 
 4   location          1615940 non-null  object 
 5   Country           1615940 non-null  object 
 6   latitude          1615940 non-null  float64
 7   longitude         1615940 non-null  float64
 8   Work Type         1615940 non-null  object 
 9   Company Size      1615940 non-null  int64  
 10  Job Posting Date  1615940 non-null  object 
 11  Preference        1615940 non-null  object 
 12  Contact Person    1615940 non-null  object 
 13  Contact           1615940 non-null  object 
 14  Job Title         1615940 non-null  object 
 15  Role              1615940 non-null  object 
 16  

Fitur yang akan diolah

1. Job Title
2. Job Description
3. skills
4. Responsibilities
5. Qualifications
6. location
7. Work Type

Kolom Unik

In [8]:
len(df['Job Title'].unique())

147

In [9]:
len(df['skills'].unique())

376

In [10]:
print(df['Qualifications'].unique()) # Pendidikan
print(len(df['Qualifications'].unique()))

['M.Tech' 'BCA' 'PhD' 'MBA' 'MCA' 'M.Com' 'BBA' 'B.Tech' 'B.Com' 'BA']
10


In [11]:
# print(df['Responsibilities'].unique()) # Pengalaman
print(len(df['Responsibilities'].unique()))

375


In [12]:
print(df['location'].unique()) # Lokasi
print(len(df['location'].unique()))

['Douglas' 'Ashgabat' 'Macao' 'Porto-Novo' 'Santiago' 'Brussels'
 'George Town' 'SÃ£o TomÃ©' 'Male' "Saint John's" 'Manama'
 'The City of Hamilton' 'Kingston' 'Banjul' 'Damascus' 'Sanaa'
 'San Marino' 'Papeete' 'Seoul' 'Kinshasa' 'Baku' 'Funafuti' 'Asmara'
 'Tegucigalpa' 'Road Town' 'Havana' 'Budapest' 'AsunciÃ³n' 'Amman' 'Abuja'
 'Tripoli' 'Canberra' 'Noumea' 'Muscat' 'Gaza' 'Honiara' 'Yamoussoukro'
 'Quito' 'Charlotte Amalie' 'Yaounde' 'Moroni' 'Madrid' 'Belmopan'
 'Valletta' 'Panama City' 'Brazzaville' 'Cockburn Town' 'Phnom Penh'
 'Monaco' 'Yerevan' 'Hagatna' 'Castries' 'New Delhi' 'Mogadishu'
 'Islamabad' 'Bandar Seri Begawan' 'Roseau' 'London' 'Kabul' 'Ulaanbaatar'
 'Bratislava' 'Rabat' 'Pretoria' 'Gaborone' 'Marigot' 'Riyadh' 'Dhaka'
 'Kathmandu' 'Kuala Lumpur' 'Bishkek' 'Sri Jayawardenepura Kotte'
 'Mexico City' 'Chisinau' 'Guatemala City' 'Moscow' 'Riga' 'Berlin'
 'Djibouti (city)' 'Paramaribo' 'City of Port Louis' 'Bamako' 'San Jose'
 'Jakarta' 'Suva' 'Kigali' 'Juba' 'Vientia

In [13]:
print(df['Work Type'].unique()) # Jenis Pekerjaan
print(len(df['Work Type'].unique()))

['Intern' 'Temporary' 'Full-Time' 'Contract' 'Part-Time']
5


## Summary

1. Total data ada 1juta lebih, tetapi yang akan digunakan hanya sample yaitu 10 ribu saja.
2. yang akan digunakan ada 7 yaitu :
  1. Job Title
  2. Job Description
  3. skills
  4. Responsibilities
  5. Qualifications
  6. location
  7. Work Type

# Cleaning

- Filter hanya 7 fitur seperti yang di EDA
- Hilangkan Null Jika ada
- Hilangkan Duplikat jika ada
- Buat jadi sample 10k data

Sampling dulu jadi 10k data

In [14]:
df_sample = df.sample(n=10000, random_state=27)
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1541831 to 1559380
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job Id            10000 non-null  int64  
 1   Experience        10000 non-null  object 
 2   Qualifications    10000 non-null  object 
 3   Salary Range      10000 non-null  object 
 4   location          10000 non-null  object 
 5   Country           10000 non-null  object 
 6   latitude          10000 non-null  float64
 7   longitude         10000 non-null  float64
 8   Work Type         10000 non-null  object 
 9   Company Size      10000 non-null  int64  
 10  Job Posting Date  10000 non-null  object 
 11  Preference        10000 non-null  object 
 12  Contact Person    10000 non-null  object 
 13  Contact           10000 non-null  object 
 14  Job Title         10000 non-null  object 
 15  Role              10000 non-null  object 
 16  Job Portal        10000 non-null  obj

7 Fitur

In [15]:
df_clean = df_sample.copy()

In [16]:
df_clean = df_clean[['Job Title', 'Job Description', 'skills', 'Responsibilities', 'Qualifications', 'location', 'Work Type']]

In [17]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1541831 to 1559380
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job Title         10000 non-null  object
 1   Job Description   10000 non-null  object
 2   skills            10000 non-null  object
 3   Responsibilities  10000 non-null  object
 4   Qualifications    10000 non-null  object
 5   location          10000 non-null  object
 6   Work Type         10000 non-null  object
dtypes: object(7)
memory usage: 625.0+ KB


Hapus Duplikat

In [18]:
df_clean.duplicated().sum()

np.int64(19)

In [19]:
df_clean = df_clean.drop_duplicates()
df_clean.duplicated().sum()

np.int64(0)

In [20]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9981 entries, 1541831 to 1559380
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job Title         9981 non-null   object
 1   Job Description   9981 non-null   object
 2   skills            9981 non-null   object
 3   Responsibilities  9981 non-null   object
 4   Qualifications    9981 non-null   object
 5   location          9981 non-null   object
 6   Work Type         9981 non-null   object
dtypes: object(7)
memory usage: 623.8+ KB


Jika Null Hapus

In [21]:
df_clean.isnull().sum()

,0
Job Title,0
Job Description,0
skills,0
Responsibilities,0
Qualifications,0
location,0
Work Type,0


## Sumary

1. 7 Fitur yang digunakan
2. Terdapat Duplicate sebanyak (19) dan dihapus
3. tidak ada data yang null

In [22]:
df_clean.head()

,Job Title,Job Description,skills,Responsibilities,Qualifications,location,Work Type
1541831,Data Entry Clerk,An Administrative Assistant provides administr...,Office management Administrative tasks Communi...,"Provide administrative support, such as managi...",PhD,Monaco,Temporary
1423110,Network Engineer,"Wireless Network Engineers design, implement, ...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",BCA,Madrid,Full-Time
788976,Structural Engineer,A Design Engineer creates and develops product...,Engineering design CAD software proficiency Pr...,Design structural systems and components for b...,MCA,San Salvador,Part-Time
233945,QA Analyst,Performance Testing Specialists assess the per...,Performance testing Load testing Stress testin...,Focus on performance and load testing to asses...,B.Tech,Niamey,Temporary
626013,Family Nurse Practitioner,Geriatric Nurse Practitioners focus on the hea...,Geriatric healthcare Geriatric assessment Geri...,"Focus on elderly patient care, addressing age-...",BA,Bangkok,Part-Time


In [44]:
df_clean['Job Title'].value_counts()

,count
Job Title,
UX/UI Designer,300
Software Engineer,176
Digital Marketing Specialist,175
Network Engineer,147
Procurement Manager,146
...,...
Sales Associate,18
Investment Advisor,17
Event Manager,17


# Modeling

In [23]:
df_modeling = df_clean.copy()

## Data  Preprosessing

In [24]:
df_modeling = df_modeling.reset_index(drop=True)

In [25]:
df_modeling['combined_text'] = (
  df_modeling['skills'] + ' ' +
  df_modeling['Job Description'] + ' ' +
  df_modeling['Responsibilities'] + ' ' +
  df_modeling['Qualifications'] + ' ' +
  df_modeling['location'] + ' ' +
  df_modeling['Work Type']
)

In [26]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'\d+', '', text)  # hapus angka
  text = re.sub(r'[^\w\s]', '', text)  # hapus tanda baca
  return text

df_modeling['combined_text'] = df_modeling['combined_text'].apply(clean_text)

Ingin prediksi `Work Type`

In [27]:
label_encoder = LabelEncoder()
df_modeling['Work Type Encoded'] = label_encoder.fit_transform(df_modeling['Work Type'])

## Content Based Filtering

In [28]:
# TF-IDF Vectirization
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf.fit_transform(df_modeling['combined_text'])

In [29]:
# Cosine Similarty
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [30]:
# Lowercase untuk pencarian job title
df_modeling['Job Title Lower'] = df_modeling['Job Title'].str.lower()

# Mapping index berdasarkan job title lowercase
indices = pd.Series(df_modeling.index, index=df_modeling['Job Title Lower']).drop_duplicates()

In [31]:
# Fungsi Rekomendasi
# indices = pd.Series(df_modeling.index, index=df_modeling['Job Title']).drop_duplicates()
def recommend_jobs(job_title, top_n=5):
  # Cari job title yang mengandung kata kunci, dalam df_modeling
  matches = df_modeling[df_modeling['Job Title'].str.lower().str.contains(job_title.lower())]

  if matches.empty:
    return "Job Title tidak ditemukan."

  # Ambil index posisi (0–9999) dari hasil match (bukan index asli dari dataset awal)
  idx = matches.index[0]

  # Hitung similarity terhadap semua job
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Ambil top_n, skip dirinya sendiri (index ke-0)
  sim_scores = sim_scores[1:top_n+1]
  job_indices = [i[0] for i in sim_scores]
  scores = [round(score * 100, 2) for _, score in sim_scores]

  # Ambil data dari df_modeling
  result = df_modeling.iloc[job_indices][['Job Title', 'location', 'Work Type']].copy()
  result['Similarity (%)'] = scores

  return result

In [32]:
df_modeling = df_modeling.reset_index(drop=True)

In [33]:
recommend_jobs('Software Engine')

,Job Title,location,Work Type,Similarity (%)
4479,Software Engineer,Belgrade,Intern,97.77
4484,Software Engineer,Harare,Contract,97.68
2129,Software Engineer,Ankara,Temporary,97.63
9723,Software Engineer,Belgrade,Part-Time,97.57
5769,Software Engineer,Prague,Part-Time,97.56


# Evaluation

## Pressision@k

In [62]:
ground_truth = {
  'software engineer': ['software engineer', 'software developer'],
  'data analyst': ['data analyst', 'business analyst'],
  'marketing manager': ['marketing manager', 'digital marketing'],
  'project manager': ['project manager'],
  'ui/ux': ['ui/ux', 'Designer', 'ui/ux designer'],
  'sales': ['sales', 'sales associat']
}

In [39]:
def precision_at_k(recommended, relevant, k=5):
  recommended_titles = recommended['Job Title'].str.lower().tolist()
  relevant_lower = [r.lower() for r in relevant]
  hits = sum(1 for title in recommended_titles if any(rel in title for rel in relevant_lower))
  return hits / k

In [63]:
test_queries = ['software engineer', 'data analyst', 'marketing manager', 'project manager', 'ui/ux', 'data scientist']
precisions = []

for query in test_queries:
  recs = recommend_jobs(query, top_n=5)
  if isinstance(recs, str):
    print(f"Query '{query}' tidak ditemukan.")
    continue
  prec = precision_at_k(recs, ground_truth.get(query, []))
  print(f"Precision@5 untuk '{query}': {prec:.2f}")
  precisions.append(prec)

print(f"Rata-rata Precision@5: {sum(precisions)/len(precisions):.2f}")


Precision@5 untuk 'software engineer': 1.00
Precision@5 untuk 'data analyst': 1.00
Precision@5 untuk 'marketing manager': 1.00
Precision@5 untuk 'project manager': 1.00
Query 'ui/ux' tidak ditemukan.
Precision@5 untuk 'data scientist': 0.00
Rata-rata Precision@5: 0.80


## Coverage

In [64]:
valid = 0
for query in test_queries:
    recs = recommend_jobs(query, top_n=5)
    if not isinstance(recs, str):
        valid += 1
coverage = valid / len(test_queries)
print(f"Coverage: {coverage*100:.2f}%")

Coverage: 83.33%



# Metrik Evaluasi

1. **Precision\@k**
   Precision\@k mengukur proporsi pekerjaan yang relevan di antara *k* rekomendasi teratas yang diberikan oleh sistem.
   Rumusnya:

   $$\text{Precision@k} = \frac{\text{Jumlah rekomendasi relevan dalam k teratas}}{k}$$

   Metrik ini menilai ketepatan rekomendasi yang disajikan kepada pengguna, dimana nilai lebih tinggi menunjukkan relevansi yang lebih baik.

2. **Coverage**
   Coverage mengukur proporsi query pencarian pekerjaan yang berhasil diberikan rekomendasi oleh sistem dari seluruh query yang diuji.
   Rumusnya:

   $$\text{Coverage} = \frac{\text{Jumlah query dengan rekomendasi valid}}{\text{Total query}} \times 100\%$$

   Metrik ini menilai sejauh mana sistem mampu memberikan rekomendasi untuk berbagai jenis pekerjaan yang dicari pengguna.


   ### Kesimpulan

Model rekomendasi pekerjaan berbasis content-based filtering ini telah berhasil memberikan rekomendasi yang relevan dengan tingkat akurasi tinggi untuk sebagian besar query. Coverage sebesar 83.33% menunjukkan bahwa sistem cukup luas dalam menyediakan rekomendasi untuk berbagai jenis pekerjaan. Meskipun demikian, ada ruang perbaikan terutama pada jenis pekerjaan dengan data lebih terbatas atau query yang kurang tepat, yang dapat menjadi fokus pengembangan selanjutnya untuk meningkatkan cakupan dan akurasi rekomendasi.
